In [16]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [ ]:
drive_time_matrix = pd.read_csv('DRIVE_matrix_duration_matrix.csv', index_col=0)
# Convert drive time matrix values from seconds to minutes (except index)
drive_time_matrix = drive_time_matrix.astype('float64') / 60
caregivers = pd.read_csv('caregivers.csv', index_col=0)
tasks = pd.read_csv('hemtjanst_tasks.csv', index_col=0)

display(drive_time_matrix)
display(tasks)

,0,156,174,269,272,273,305,396,406,457,...,1687,1690,1691,1694,1695,1704,1709,1712,1714,1715
0,0.000000,4.800000,9.033333,7.633333,7.200000,4.533333,7.166667,7.200000,8.633333,12.583333,...,10.450000,9.400000,5.766667,10.983333,10.066667,11.500000,13.066667,5.933333,13.466667,5.733333
156,4.366667,0.000000,9.183333,7.783333,7.350000,4.700000,7.316667,7.350000,8.783333,12.750000,...,10.600000,9.566667,5.933333,11.150000,10.216667,11.650000,13.216667,6.083333,13.616667,5.883333
174,7.316667,8.933333,0.000000,7.183333,8.883333,7.316667,5.866667,4.250000,2.066667,14.350000,...,12.200000,11.166667,5.450000,11.533333,11.816667,9.550000,11.116667,6.633333,7.300000,8.516667
269,6.833333,8.450000,7.366667,0.000000,8.566667,6.833333,5.533333,5.933333,6.966667,14.016667,...,11.866667,10.833333,6.616667,11.200000,11.500000,8.183333,9.750000,7.800000,9.533333,8.033333
272,5.450000,7.066667,8.966667,5.750000,0.000000,4.400000,4.766667,5.683333,8.566667,7.133333,...,5.266667,6.850000,7.000000,4.366667,4.883333,9.133333,10.700000,7.166667,10.483333,6.966667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1704,10.283333,11.916667,8.450000,8.083333,9.550000,9.233333,8.650000,7.000000,8.050000,15.000000,...,12.866667,11.816667,9.733333,12.200000,12.483333,0.000000,3.233333,10.916667,6.433333,11.150000
1709,11.900000,13.516667,10.050000,9.700000,11.166667,10.833333,10.266667,8.616667,9.666667,16.616667,...,14.466667,13.433333,11.350000,13.800000,14.100000,3.716667,0.000000,12.516667,8.033333,12.766667
1712,4.833333,6.466667,6.533333,7.433333,7.500000,4.850000,6.866667,6.900000,7.416667,12.900000,...,10.750000,9.716667,0.816667,11.300000,10.366667,11.300000,12.866667,0.000000,13.183333,4.866667
1714,11.700000,13.316667,7.166667,9.233333,10.683333,10.366667,10.250000,8.133333,6.766667,16.150000,...,14.000000,12.966667,11.483333,13.333333,13.616667,6.716667,8.283333,12.666667,0.000000,12.900000


,StartTime,EndTime,TaskType,CaregiverID,RequiresWoman,HasPet
ClientID,,,,,,
1661.0,2025-03-05T07:30:00+01:00,2025-03-05T07:50:00+01:00,Hemtjänst,1146,True,False
679.0,2025-03-05T07:30:00+01:00,2025-03-05T08:00:00+01:00,Hemtjänst,33,False,False
1569.0,2025-03-05T07:30:00+01:00,2025-03-05T08:15:00+01:00,Hemtjänst,1180,False,False
1612.0,2025-03-05T07:30:00+01:00,2025-03-05T07:40:00+01:00,Hemtjänst,1163,False,False
1474.0,2025-03-05T07:30:00+01:00,2025-03-05T08:00:00+01:00,Hemtjänst,1284,False,False
...,...,...,...,...,...,...
1704.0,2025-03-05T22:10:00+01:00,2025-03-05T22:30:00+01:00,Hemtjänst,1364,False,False
1349.0,2025-03-05T22:10:00+01:00,2025-03-05T22:20:00+01:00,Hemtjänst,1264,False,False
978.0,2025-03-05T22:10:00+01:00,2025-03-05T22:25:00+01:00,Hemtjänst,979,False,False


In [25]:
# Convert time strings to datetime objects and extract minutes since midnight
tasks['start_minutes'] = pd.to_datetime(tasks['StartTime']).dt.hour * 60 + pd.to_datetime(tasks['StartTime']).dt.minute
tasks['end_minutes'] = pd.to_datetime(tasks['EndTime']).dt.hour * 60 + pd.to_datetime(tasks['EndTime']).dt.minute

tasks

,StartTime,EndTime,TaskType,CaregiverID,RequiresWoman,HasPet,start_seconds,end_seconds,start_minutes,end_minutes
ClientID,,,,,,,,,,
1661.0,2025-03-05T07:30:00+01:00,2025-03-05T07:50:00+01:00,Hemtjänst,1146,True,False,450,470,450,470
679.0,2025-03-05T07:30:00+01:00,2025-03-05T08:00:00+01:00,Hemtjänst,33,False,False,450,480,450,480
1569.0,2025-03-05T07:30:00+01:00,2025-03-05T08:15:00+01:00,Hemtjänst,1180,False,False,450,495,450,495
1612.0,2025-03-05T07:30:00+01:00,2025-03-05T07:40:00+01:00,Hemtjänst,1163,False,False,450,460,450,460
1474.0,2025-03-05T07:30:00+01:00,2025-03-05T08:00:00+01:00,Hemtjänst,1284,False,False,450,480,450,480
...,...,...,...,...,...,...,...,...,...,...
1704.0,2025-03-05T22:10:00+01:00,2025-03-05T22:30:00+01:00,Hemtjänst,1364,False,False,1330,1350,1330,1350
1349.0,2025-03-05T22:10:00+01:00,2025-03-05T22:20:00+01:00,Hemtjänst,1264,False,False,1330,1340,1330,1340
978.0,2025-03-05T22:10:00+01:00,2025-03-05T22:25:00+01:00,Hemtjänst,979,False,False,1330,1345,1330,1345


In [21]:
# Filter drivers who use car as mode of transport
car_caregivers = caregivers[caregivers['ModeOfTransport'] == 'car']

# Get the list of driver IDs who use cars
car_driver_ids = car_caregivers.index.tolist()

# Filter tasks to only include those assigned to car caregivers
car_tasks = tasks[tasks['CaregiverID'].isin(car_driver_ids) & (tasks['TaskType'] == 'Hemtjänst')]

# Display the results
print(f"Number of car caregivers: {len(car_caregivers)}")
print(f"Number of tasks assigned to car caregivers: {len(car_tasks)}")

# Display the first few rows of each dataframe
display(car_caregivers)
display(car_tasks)

Number of car caregivers: 17
Number of tasks assigned to car caregivers: 155


,ModeOfTransport,IsMan,IsAllergic
ID,,,
979,car,True,True
36,car,False,False
33,car,False,False
949,car,True,False
438,car,False,False
284,car,False,False
1108,car,True,False
1173,car,False,False
113,car,False,True


,StartTime,EndTime,TaskType,CaregiverID,RequiresWoman,HasPet,start_seconds,end_seconds
ClientID,,,,,,,,
1661.0,2025-03-05T07:30:00+01:00,2025-03-05T07:50:00+01:00,Hemtjänst,1146,True,False,450,470
679.0,2025-03-05T07:30:00+01:00,2025-03-05T08:00:00+01:00,Hemtjänst,33,False,False,450,480
1366.0,2025-03-05T07:30:00+01:00,2025-03-05T08:15:00+01:00,Hemtjänst,949,False,False,450,495
1349.0,2025-03-05T07:30:00+01:00,2025-03-05T07:40:00+01:00,Hemtjänst,1108,False,False,450,460
1298.0,2025-03-05T07:30:00+01:00,2025-03-05T08:00:00+01:00,Hemtjänst,958,True,False,450,480
...,...,...,...,...,...,...,...,...
1649.0,2025-03-05T22:00:00+01:00,2025-03-05T22:20:00+01:00,Hemtjänst,438,False,False,1320,1340
396.0,2025-03-05T22:05:00+01:00,2025-03-05T22:35:00+01:00,Hemtjänst,833,False,False,1325,1355
1704.0,2025-03-05T22:10:00+01:00,2025-03-05T22:30:00+01:00,Hemtjänst,1364,False,False,1330,1350


In [29]:
import gurobipy as gp
from gurobipy import GRB

V = tasks.index.tolist()
K = car_caregivers.index.tolist()
c = drive_time_matrix.to_dict()

e = tasks['start_minutes'].to_dict()
l = tasks['end_minutes'].to_dict()
s = (tasks['end_minutes'] - tasks['start_minutes']).to_dict()

p = {}
for i in V:
    for k in K:
        p[i, k] = 1
        # Check if task requires woman and caregiver is a man
        if tasks.loc[i, 'RequiresWoman'].item() and car_caregivers.loc[k, 'IsMan'].item():
            p[i, k] = 0
        # Check if client has pet and caregiver is allergic
        if tasks.loc[i, 'HasPet'].item() and car_caregivers.loc[k, 'IsAllergic'].item():
            p[i, k] = 0

model = gp.Model("home_care_scheduling")

x = model.addVars(V, V, K, vtype=GRB.BINARY, name="x")
t = model.addVars(V, K, vtype=GRB.CONTINUOUS, name="t")
w = model.addVars(K, vtype=GRB.CONTINUOUS, name="w")

# Objective function: minimize total downtime
model.setObjective(gp.quicksum(w[k] for k in K), GRB.MINIMIZE)

# Constraints
# Each patient is visited exactly once
model.addConstrs((gp.quicksum(x[i, j, k] for k in K for j in V) == 1 for i in V), name="visit_once")

# Flow conservation constraints
model.addConstrs((gp.quicksum(x[i, j, k] for j in V) - gp.quicksum(x[j, i, k] for j in V) == 0 for k in K for i in V), name="flow_conservation")

# Each caregiver starts and ends at the HQ
model.addConstrs((gp.quicksum(x[0, j, k] for j in V) == gp.quicksum(x[j, len(V)-1, k] for j in V) for k in K), name="start_end_hq")

# Caregiver qualifications
model.addConstrs((x[i, j, k] <= p[j, k] for i in V for j in V for k in K), name="qualifications")

# Time window constraints
model.addConstrs((t[i, k] >= e[i] for i in V for k in K), name="earliest_start")
model.addConstrs((t[i, k] <= l[i] - s[i] for i in V for k in K), name="latest_start")

# Travel time constraints
M = 1e6  # A sufficiently large constant
model.addConstrs((t[j, k] >= t[i, k] + c[i][j] + s[i] - M * (1 - x[i, j, k]) for i in V for j in V if i != 0 for k in K), name="travel_time")

# Downtime constraints
model.addConstrs((w[k] >= t[len(V)-1, k] - t[0, k] - gp.quicksum(s[i] * gp.quicksum(x[j, i, k] for j in V) for i in V) for k in K), name="downtime")

# Optimize the model
model.optimize()

ValueError: can only convert an array of size 1 to a Python scalar